In [3]:
import os
import melee 
import numpy as np
import pandas as pd
from data_processing import process_slp_file, save_processed_data

In [2]:
os.getcwd()

'/Users/alexoon/Desktop/smash_bot/icl_smash'

In [6]:
# Process a single file
slp_file = "./data/Stream-Game_20220828T223051.slp"
sampled_data, columns = process_slp_file(slp_file)
sampled_data.shape

# Save the results
# save_processed_data(sampled_data, columns, "./training_data", "DESIRED_FILENAME")

# Or use the main function directly
# shape = main(slp_file, "./training_data", "DESIRED_FILENAME")

(842, 97)

In [7]:
# This is the other data from the other github repo
alt_data = np.load("/Users/alexoon/Desktop/smash_bot/icl_smash/training_data/CPTFALCON_FOX_on_FINAL_DESTINATION_data_X.npy")
alt_data.shape

(401, 37)